In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# PPO_EKF_25

|||
|-|-|
| randomize_offset | false |
| horizon | 32768 |
| reward_fn | rw_distance_normalized |
| map | Town01 |

In [ ]:
# time, frame1_x, frame1_y, frame2_x, frame2_y, ...
trajectories_25 = pd.read_csv('data/PPO_EKF_25/trajectories.csv')
trajectories_25.sort_values(by='time', inplace=True)
trajectories_25['time'] -= trajectories_25['time'].min()
trajectories_25.head()

In [ ]:
# Separate frames
gt_25 = trajectories_25[['time', 'EGO_1/IMU_x', 'EGO_1/IMU_y']].rename(columns={'EGO_1/IMU_x': 'x', 'EGO_1/IMU_y': 'y'})

EKF_25 = trajectories_25[['time', 'base_link_x', 'base_link_y']].rename(columns={'base_link_x': 'x', 'base_link_y': 'y'})
EKF_25['error_x'] = EKF_25['x'] - gt_25['x']
EKF_25['error_y'] = EKF_25['y'] - gt_25['y']
EKF_25['error'] = np.sqrt(EKF_25['error_x']**2 + EKF_25['error_y']**2)

model_25 = trajectories_25[['time', 'model_frame_x', 'model_frame_y']].rename(columns={'model_frame_x': 'x', 'model_frame_y': 'y'})
model_25['error_x'] = model_25['x'] - gt_25['x']
model_25['error_y'] = model_25['y'] - gt_25['y']
model_25['error'] = np.sqrt(model_25['error_x']**2 + model_25['error_y']**2)

NameError: name 'trajectories_25' is not defined

In [ ]:
# Plot trajectories over specified horizon
t0 = 0.0
t1 = 10.0
mask = (gt_25['time'] >= t0) & (gt_25['time'] <= t1)

plt.figure(figsize=(10, 6))
plt.plot(gt_25['x'][mask], gt_25['y'][mask], label='Ground Truth', color='green', linestyle='--')
plt.plot(EKF_25['x'][mask], EKF_25['y'][mask], label='EKF', color='blue')
plt.plot(model_25['x'][mask], model_25['y'][mask], label='Model', color='orange')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Trajectories Over Time')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plot errors over time
t0 = 0.0
t1 = 10.0
mask = (gt_25['time'] >= t0) & (gt_25['time'] <= t1)

plt.figure(figsize=(10, 6))
plt.plot(EKF_25['time'][mask], EKF_25['error'][mask], label='EKF', color='blue')
plt.plot(model_25['time'][mask], model_25['error'][mask], label='Model', color='orange')
plt.xlabel('Time (s)')
plt.ylabel('Error (m)')
plt.title('Prediction error over time')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plot a heatmap of errors
plt.figure(figsize=(10, 8))
hb = plt.hexbin(
    gt_25['x'], gt_25['y'],
    C=model_25['error'],
    gridsize=60,
    reduce_C_function=np.mean,
    cmap='inferno'
)
plt.colorbar(hb, label='Error (m)')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Spatial heatmap of model error')
plt.grid()
plt.show()

In [ ]:
# Plot a heatmap of errors std
plt.figure(figsize=(10, 8))
hb = plt.hexbin(
    gt_25['x'], gt_25['y'],
    C=model_25['error'],
    gridsize=60,
    reduce_C_function=np.std,
    cmap='inferno'
)
plt.colorbar(hb, label='Error (m)')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Spatial heatmap of model error std')
plt.grid()
plt.show()